In [1]:
# Currently working on:
# check if mana pool contains mana
# how to check playing multiple spells in a turn with over lapping error

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
from collections import Counter
from collections.abc import Iterable

In [4]:
from typing import List
from abc import ABC

In [134]:
from random import choice, shuffle

In [135]:
a = set(['W' , 'G'])
b = set(['W', 'G', 'B'])
a.symmetric_difference(b)

{'B'}

In [136]:
import re

In [137]:
string = 'W/GUB/WGG'
colours = []

i = 0
while i < len(string) - 1:
    if string[i + 1] == '/':
        colours += [string[i] + string[i+2]]
        i += 3
    else:
        colours += [string[i]]
        i += 1

if string[-2] != '/':
    colours += [string[-1]]
        
colours

['WG', 'U', 'BW', 'G', 'G']

In [138]:
gw = set('gw')
w = set('wg')

gw.intersection(w), w.intersection(gw)

({'g', 'w'}, {'g', 'w'})

In [139]:
valid_mana_symbols = set([x for x in 'WGUBRC/'])

class Mana():
    """
    5 colours - WGUBR
    C - colourless
    todo: add duel mana types
    
    colour: a single char or G/W that represents 1 mana symbol
    """
    def __init__(self, colour: str):
        if not set([x for x in colour]).issubset(valid_mana_symbols):
            raise ValueError('Invalid colour')
            
        if len(colour) != 1 + colour.count('/') * 2:
            raise ValueError('Invalid Mana')
            
        if len(colour) == 1:
            self.type = colour
        else:
            self.type = ''.join(colour.split('/'))
            
    def __eq__(self, other):
        if isinstance(other, Mana):
            if other.type == 'C':
                return True
            elif set(self.type).intersection(set(other.type)):
                return True
                
        return False
    
    def __hash__(self):
        return hash(self.type)
    
    def __repr__(self):
        return 'Mana(' + str('/'.join([x for x in self.type])) + ')'
    

class ManaPool():
    
    def __init__(self):
        self.mana = Counter()
        
    def add_mana(self, mana_str: List[Mana]):
        if isinstance(mana_str, str):
            if len(mana_str) < 3:
                self.mana += Counter([Mana(x) for x in mana_str])
                return
            
            mana = []
            i = 0
            while i < len(mana_str) - 1:
                if mana_str[i + 1] == '/':
                    mana += [mana_str[i:i+3]]
                    i += 3
                else:
                    mana += [mana_str[i]]
                    i += 1

            if mana_str[-2] != '/':
                mana += [mana_str[-1]]
            
            # Convert to mana objects
            for i, m in enumerate(mana):
                mana[i] = Mana(m)
            
            self.mana += Counter(mana)
            
        elif isinstance(mana_str, Iterable):
            # Convert to mana objects
            for i, m in enumerate(mana):
                if isinstance(m, str):
                    mana[i] = Mana(m)
                else:
                    mana[i] = m
                
            self.mana += Counter(mana)
        
        elif isinstance(mana_str, Mana):
            self.mana += Counter([mana_str])
        
        else:
            raise ValueError('Invalid mana representation')
        
    def remove_mana(self, mana_str: List[Mana]):
        if isinstance(mana_str, str):
            if len(mana_str) < 3:
                self.mana -= Counter([Mana(x) for x in mana_str])
                return
            
            mana = []
            i = 0
            while i < len(mana_str) - 1:
                if mana_str[i + 1] == '/':
                    mana += [mana_str[i:i+3]]
                    i += 3
                else:
                    mana += [mana_str[i]]
                    i += 1

            if mana_str[-2] != '/':
                mana += [mana_str[-1]]
            
            # Convert to mana objects
            for i, m in enumerate(mana):
                mana[i] = Mana(m)
            
            self.mana -= Counter(mana)
            
        elif isinstance(mana_str, Iterable):
            # Convert to mana objects
            for i, m in enumerate(mana):
                mana[i] = Mana(m)
                
            self.mana -= Counter(mana_str)
            
        elif isinstance(mana_str, Mana):
            self.mana += Counter([mana_str])
        
        else:
            raise ValueError('Invalid mana representation')
        
    def clear(self):
        self.mana.clear()
        
    def __iadd__(self, mana):
        self.add_mana(mana)
        return self
        
    def __isub__(self, mana):
        self.remove_mana(mana)
        return self
        
    def __contains__(self, mana):
        return mana in self.mana
    
    def __repr__(self):
        res = "ManaPool("
        
        for mana, amount in self.mana.items():
            res += str(mana) + ": " + str(amount) + ", "
        
        if len(self.mana) > 0:
            res = res[:-2]
        
        return res + ')'

In [288]:
class Card():
    def __init__(self, cost: Counter):
        if cost != None:
            self.cost = Counter(cost)
        else:
            self.cost = None
            
        self.tapped = False

    def tap(self):
        self.tapped = True
        
    def untap(self):
        self.tapped = False
        
    def etb(self):
        pass
        
    def is_playable(self,  available_mana: ManaPool):
        return self.cost in available_mana
    
    def __repr__(self):
        return 'Card (' + str(self.cost) + ')'
    
class ManaDork(Card):
    def __init__(self, cost, creates):
        self.creates = creates
        
    def tap(self):
        return Mana(self.creates)
    
    
class Explore(Card):
    def __init__(self, cost, num):
        pass
    
    def etb(self):
        pass
    
class Surveil(Card):
    def __init__(self):
        pass
    
class Draw():
    def __init__(self, cost, num):
        pass
        
        
class Board():
    pass

class Deck():
    def __init__(self):
        self.deck = []
    
    def __iadd__(self, cards):
        self.deck += cards
        shuffle(self.deck)
        return self
        
    def draw(self):
        return self.deck.pop()
    
    def __len__(self):
        return len(self.deck)
    
class Hand():
    pass

In [312]:
basic_lands_types = {'G': 'Forest', 'W': 'Plains', 'U': 'Island', 'B': 'Swamp', 'R': 'Mountain', 'C': 'Wastes'}

class Land(ABC, Card):
    
    def __init__(self, colours: List[Mana]):
        self.colours = colours
        self.types = []
        
        # TODO: Add a cannot instantiate clause here
        super().__init__([])
        
    def is_type(self, land_type):
        return land_type in self.types
        
    def __eq__(self, other):
        if isinstance(other, Land):
            return set(self.colours) == set(other.colours)

        return False

    def tap(self):
        return Mana('/'.join([c for c in self.colours]))
#         return NotImplemented()
    
    def __repr__(self):
        return NotImplemented()
    
    
class BasicLand(Land):
    def __init__(self, colour: str):
        if len(colour) > 1:
            raise ValueError('Basics can only produce 1 type')
        
        super().__init__([colour])
        
        self.types += [basic_lands_types[colour]]
    
    def __repr__(self):
        return 'BasicLand(' + self.colours[0] + ')'
    
    def __eq__(self, other):
        if isinstance(other, BasicLand):
            return set(self.colours) == set(other.colours)
        
        return False
        
        
class CheckLand(Land):
    
    def __init__(self, colours: List[Mana]):
        super().__init__(colours)
        
        self.tapped = True
                
    def etb(self, currentLands: List[Land]):
        # Maybe expand this is a loop?
        if any(any([land.is_type(basic_lands_types[colour]) for colour in self.colours]) for land in lands):
            self.tapped = False    
            
    def __repr__(self):
        return "CheckLand(" + ''.join(self.colours) + ")"
            
    def __eq__(self):
        if isinstance(other, CheckLand):
            return set(self.colours) == set(other.colours)
        
        return False
        
        
class ShockLand(Land):
    
    def __init__(self, colours: List[Mana]):
        super().__init__(colours)
        
        self.types += [basic_lands_types[c] for c in colours]
        
    def __eq__(self, other):
        if isinstance(other, ShockLand):
            if set(self.colours) == set(other.colour):
                return True
            
        return False
    
    def __repr__(self):
        return 'ShockLand(' + ''.join(self.colours) + ")"

## Class Testing

### Test mana

In [313]:
# Check mana types
Mana('G')
Mana('G/W')
Mana('G/W/U')

# Check colours are equal
white_mana = Mana('W')
assert white_mana == Mana('W')

# Check colours are not equal
green_mana = Mana('G')
assert white_mana != green_mana

colourless_mana = Mana('C')
# Check colourless is colourless
assert colourless_mana == Mana('C')
# Check coloured is colourless
assert Mana('G') == Mana('C')
# Check colourless is not coloured
assert colourless_mana != Mana('G')

try:
    mana = Mana('Q')
except ValueError as e:
    assert str(e) == 'Invalid colour'
    
# Check that G/W mana is G or W
green_or_white_mana = Mana('G/W')
blue_mana = Mana('U')
assert green_or_white_mana == green_mana
assert green_mana == green_or_white_mana
assert green_or_white_mana == colourless_mana
assert colourless_mana != green_or_white_mana
assert green_or_white_mana != blue_mana

# Check G/W mana is U/G
green_or_blue_mana = Mana('G/U')
assert green_or_white_mana == green_or_blue_mana
assert green_or_blue_mana == green_or_white_mana

### Mana Pool Testing

In [314]:
mana_pool = ManaPool()
mana_pool += 'WWBU'
assert str(mana_pool) == 'ManaPool(Mana(W): 2, Mana(B): 1, Mana(U): 1)'
mana_pool += 'G'
assert str(mana_pool) == 'ManaPool(Mana(W): 2, Mana(B): 1, Mana(U): 1, Mana(G): 1)'
mana_pool.add_mana('GGB')
assert str(mana_pool) == 'ManaPool(Mana(W): 2, Mana(B): 2, Mana(U): 1, Mana(G): 3)'
mana_pool -= 'GW'
assert str(mana_pool) == 'ManaPool(Mana(W): 1, Mana(B): 2, Mana(U): 1, Mana(G): 2)'
mana_pool.remove_mana('WU')
assert str(mana_pool) == 'ManaPool(Mana(B): 2, Mana(G): 2)'
mana_pool.add_mana('G/W')
assert str(mana_pool) == 'ManaPool(Mana(B): 2, Mana(G): 2, Mana(G/W): 1)'
# mana_pool.clear()
# assert str(mana_pool) == 'ManaPool()'

In [315]:
print(mana_pool)
mana_pool.mana.get(Mana('G/W'), 'None')

ManaPool(Mana(B): 2, Mana(G): 2, Mana(G/W): 1)


1

### Test Lands

In [316]:
plains = BasicLand('W')
assert plains.tap() == Mana('W')

# Test each colour
for colour in 'WGBUR':
    basic = BasicLand(colour)
    assert basic.tap() == Mana(colour)

# Test colourless
basic = BasicLand('C')
assert basic.tap() == colourless_mana

# Test invalid basic
try:
    basic = BasicLand('WG')
except ValueError as e:
    assert str(e) == 'Basics can only produce 1 type'
    
# Test check land
lands = []
sunpetal_grove = CheckLand('GW')
sunpetal_grove.etb(lands)
assert sunpetal_grove.tap() == green_or_white_mana
assert sunpetal_grove.tap() == green_mana
assert sunpetal_grove.tap() == white_mana
assert sunpetal_grove.tapped == True

# With correct basic in play
lands = [plains]
sunpetal_grove = CheckLand('GW')
sunpetal_grove.etb(lands)
assert sunpetal_grove.tap() == green_or_white_mana
assert sunpetal_grove.tap() == green_mana
assert sunpetal_grove.tap() == white_mana
assert sunpetal_grove.tapped == False

# Test shock land
temple_garden = ShockLand('GW')
temple_garden.etb()
assert temple_garden.tap() == green_or_white_mana
assert temple_garden.tap() == green_mana
assert temple_garden.tap() == white_mana
assert temple_garden.tapped == False

# Check + shock untapped check
lands = [temple_garden]
rootbound_crag = CheckLand('GR')
rootbound_crag.etb(lands)
green_or_red_mana = Mana('R/G')
red_mana = Mana('R')
assert rootbound_crag.tap() == green_or_red_mana
assert rootbound_crag.tap() == green_or_white_mana
assert rootbound_crag.tap() == green_mana
assert rootbound_crag.tap() == red_mana
assert rootbound_crag.tap() != white_mana
assert rootbound_crag.tapped == False

### Mana pool from lands

In [317]:
lands = [temple_garden, sunpetal_grove]
mana_pool = ManaPool()

for land in lands:
    mana_pool += land.tap()
    
mana_pool

ManaPool(Mana(G/W): 2)

## Deck

In [318]:
deck = Deck()

In [319]:
deck += [BasicLand('G')] * 24
deck += [Card(None)] * 32
deck += [Card([Mana('G')])] * 4

assert len(deck) >= 60

In [327]:
hand = []
lands = []
mana_pool = ManaPool()

# Draw a hand
for i in range(0, 7):
    hand += [deck.draw()]
print(hand)
    
for turn in range(0, 1):
    # draw
    hand += [deck.draw()]
    print(hand)
    
    # play a land
    for card in hand:
        if isinstance(card, Land):
            print(card)
            lands.append(card)
            hand.remove(card)
            break
    print(lands)
    
    # Check if can cast any spells
    mana_pool = [land.tap() for land in lands]
    print(mana_pool)
    
    for card in hand:
        if card.cost in mana_pool:
            print(card, '!!!')
    
    # End turn
    mana_pool.clear()
    print()

[BasicLand(G), Card (None), BasicLand(G), Card (Counter({Mana(G): 1})), Card (None), Card (None), BasicLand(G)]
[BasicLand(G), Card (None), BasicLand(G), Card (Counter({Mana(G): 1})), Card (None), Card (None), BasicLand(G), Card (None)]
BasicLand(G)
[BasicLand(G)]
[Mana(G)]



In [321]:
hand

[Card (None), Card (None), Card (None), Card (None), Card (None)]

In [331]:
card = Card([Mana('G')])
mana_pool = ManaPool()
mana_pool += 'G'
print(card, mana_pool, card.is_playable(mana_pool))

TypeError: unhashable type: 'Counter'

In [337]:
c = Counter({Mana('G') : 1})

In [338]:
c

Counter({Mana(G): 1})

In [339]:
Mana('G') in c

True

## Run 100,000 Experiments of draws

In [30]:
from collections import Counter

In [39]:
mana = Counter([x for x in 'WWUUG'])

In [40]:
cost = Counter([x for x in 'WWU'])

In [41]:
mana, cost

(Counter({'W': 2, 'U': 2, 'G': 1}), Counter({'W': 2, 'U': 1}))

In [51]:
cost - mana

Counter()

In [52]:
if not cost - mana:
    print('Playable')
else:
    print('No playable')

Playable


In [152]:
mana -= Counter('W')

In [153]:
mana

Counter({'U': 2, 'G': 1})

In [58]:
isinstance('a', str)

True